In [1]:
# Import libraries
import pandas as pd
import numpy as np
import requests
import pickle
import re
from bs4 import BeautifulSoup
from sys import stdout

# Part 1: Marvel dataset

## 1.1 Parse the Marvel characters

**We go on the page where all the `Earth-616` Marvel characters are. We start with the first page and we make a list of all the URLs for the different characters.**

**Collect the first URLs:**

In [ ]:
URL = 'https://marvel.fandom.com/wiki/Category:Earth-616_Characters'
r = requests.get(URL)
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')

# Collect url of each character in the web page
publications_wrappers = soup.find_all('li', class_='category-page__member')
list_url = []

for p in publications_wrappers:
    for a in p.find_all('a', href=True):
        list_url.append(a['href'])
        
# Remove duplicate
my_set = set(list_url)
good_list_url = list(my_set)

**We then use these URLs to collect the needed characterstics for each Marvel character.**

**Parse the first page:**

In [ ]:
# Initialization
new_columns = ['URL', 'Real Name', 'Identity', 'Current Alias', 'Citizenship',
                'Marital Status', 'Occupation',
                'Education', 'Gender', 'Height', 'Weight',
                'Eyes','Hair',
                'Place of Birth']

personnage_pd = pd.DataFrame(columns=new_columns)
idx = 0
dict_geant={}

for pers in good_list_url:
    # Get URL and use html parser
    URL_char = 'https://marvel.fandom.com' + pers
    URL_char = URL_char.replace("'","")
    r_char = requests.get(URL_char)
    page_body_char = r_char.text
    soup_char = BeautifulSoup(page_body_char, 'html.parser')
    
    # Initialize variables
    url, name, identity, current_alias, citizenship, marital_status, occupation = '','','','','','',''
    education, gender, height, weight, eyes, hair, place_of_birth = '','','','','','',''
    personnage =[]
    
    # Parsing
    url = pers
    side_tab = soup_char.find_all('div', class_='conjoined-infoboxes')
    for p in side_tab:
        for pp in p.find_all('div', class_='pi-item pi-data pi-item-spacing pi-border-color'):
            for ppp in pp.find_all('h3', class_='pi-data-label pi-secondary-font'):
                if(ppp.text[1:]=="Real Name"):
                    name = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[1:]=="Identity"):
                    identity = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[1:]=="Current Alias"):
                    current_alias = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[1:]=="Citizenship"):
                    citizenship = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[1:]=="Marital Status"):
                    marital_status = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[1:]=="Occupation"):
                    occupation = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[1:]=="Education"):
                    education = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[1:]=="Gender"):
                    gender = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[1:]=="Height"):
                    height = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[1:]=="Weight"):
                    weight = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[1:]=="Eyes"):
                    eyes = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[1:]=="Hair"):
                    hair = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[1:]=="Place of Birth"):
                    place_of_birth = pp.find('div', class_='pi-data-value pi-font').text

    characteristics_pd = pd.DataFrame([[url, name[1:], identity, current_alias, citizenship, marital_status, occupation,
                         education, gender, height, weight, eyes, hair, place_of_birth]], columns = new_columns)
    personnage_pd = personnage_pd.append(characteristics_pd,ignore_index=True)
    
nextpage = soup.find('link', {"rel" : "next"})

**After parsing the first page, we can get the links for the next pages, parsing them one by one. After that, we use the collected links to access each character's webpage exactly as we did for the first page. We then follow by storing the characterstics we need for our analysis.**

**Parse the remaining pages:**

In [ ]:
while(len(nextpage['href'])):
    urlnext_page = nextpage['href']
    r = requests.get(urlnext_page)
    page_body = r.text
    soup = BeautifulSoup(page_body, 'html.parser')

    # Collect URL of each character in the web page
    publications_wrappers = soup.find_all('li', class_='category-page__member')
    list_url = []
    for p in publications_wrappers:
        for a in p.find_all('a', href=True):
            list_url.append(a['href'])
            
    # Remove duplicate
    my_set = set(list_url)
    good_list_url = list(my_set)
    
    idx = 0
    dict_geant = {}
    
    for pers in good_list_url:
        # Get URL and use html parser
        URL_char = 'https://marvel.fandom.com' + pers
        URL_char = URL_char.replace("'","")
        r_char = requests.get(URL_char)
        page_body_char = r_char.text
        soup_char = BeautifulSoup(page_body_char, 'html.parser')
        
        # Initialize variables
        personnage =[]
        
        # Parsing
        side_tab = soup_char.find_all('div', class_='conjoined-infoboxes')
        url = pers
        for p in side_tab:
            for pp in p.find_all('div', class_='pi-item pi-data pi-item-spacing pi-border-color'):
                for ppp in pp.find_all('h3', class_='pi-data-label pi-secondary-font'):
                    if(ppp.text[1:]=="Real Name"):
                        name = pp.find('div', class_='pi-data-value pi-font').text
                    if(ppp.text[1:]=="Identity"):
                        identity = pp.find('div', class_='pi-data-value pi-font').text
                    if(ppp.text[1:]=="Current Alias"):
                        current_alias = pp.find('div', class_='pi-data-value pi-font').text
                    if(ppp.text[1:]=="Citizenship"):
                        citizenship = pp.find('div', class_='pi-data-value pi-font').text
                    if(ppp.text[1:]=="Marital Status"):
                        marital_status = pp.find('div', class_='pi-data-value pi-font').text
                    if(ppp.text[1:]=="Occupation"):
                        occupation = pp.find('div', class_='pi-data-value pi-font').text
                    if(ppp.text[1:]=="Education"):
                        education = pp.find('div', class_='pi-data-value pi-font').text
                    if(ppp.text[1:]=="Gender"):
                        gender = pp.find('div', class_='pi-data-value pi-font').text
                    if(ppp.text[1:]=="Height"):
                        height = pp.find('div', class_='pi-data-value pi-font').text
                    if(ppp.text[1:]=="Weight"):
                        weight = pp.find('div', class_='pi-data-value pi-font').text
                    if(ppp.text[1:]=="Eyes"):
                        eyes = pp.find('div', class_='pi-data-value pi-font').text
                    if(ppp.text[1:]=="Hair"):
                        hair = pp.find('div', class_='pi-data-value pi-font').text
                    if(ppp.text[1:]=="Place of Birth"):
                        place_of_birth = pp.find('div', class_='pi-data-value pi-font').text

        characteristics_pd = pd.DataFrame([[url, name[1:], identity, current_alias, citizenship, marital_status, occupation,
                          education, gender, height, weight, eyes, hair, place_of_birth]], columns = new_columns)
        personnage_pd = personnage_pd.append(characteristics_pd,ignore_index=True)
        
        nextpage = soup.find('link', {"rel" : "next"})

**Since the parsing takes a relatively high amount of time, it's better to save it to pickle.**

**Save file to pickle:**

In [ ]:
pickle.dump(personnage_pd, open('characters_marvel.txt','wb'))

**Open the saved file:**

In [2]:
with open('characters_marvel.txt', 'rb') as f:
    characters_marvel = pickle.load(f)

characters_marvel

URL                         Real Name  \
0      /wiki/Aaron_the_Aakon_(Earth-616)                             Aaron   
1                  /wiki/2-D_(Earth-616)  Darell (full name unrevealed)[1]   
2      /wiki/Abraham_Erskine_(Earth-616)                Abraham Erskine[1]   
3              /wiki/11-Ball_(Earth-616)                            nknown   
4              /wiki/Abraham_(Earth-616)                           Abraham   
...                                  ...                               ...   
28004      /wiki/Zyziwc_Tiel_(Earth-616)                       Zyziwc Tiel   
28005            /wiki/Zzzax_(Earth-616)                             Zzzax   
28006        /wiki/%C3%89riu_(Earth-616)                           Ériu[1]   
28007             /wiki/Zyro_(Earth-616)                              Zyro   
28008           /wiki/Zzutak_(Earth-616)                            Zzutak   

                            Identity         Current Alias       Citizenship  \
0                    Secret Identity                                   Aakon   
1                    Secret Identity                   2-D          American   
2      Known to Authorities Identity  Dr. Joseph Reinstein  German, American   
3                    Secret Identity               11-Ball          American   
4                   No Dual Identity                                           
...                              ...                   ...               ...   
28004                                                 Nova                     
28005               No Dual Identity                                           
28006                                                                          
28007               No Dual Identity                             Kree Empire   
28008               No Dual Identity                                           

      Marital Status                              Occupation  \
0             Single                            Slave trader   
1             Single                              Adventurer   
2            Married                               Scientist   
3             Single  Professional criminal; former henchman   
4            Married                                 Prophet   
...              ...                                     ...   
28004                                               Corpsman   
28005         Single                 Purveyor of destruction   
28006                                                          
28007                                    Starship chief-tech   
28008                                                Servent   

                     Education   Gender                                Height  \
0                                  Male                                         
1                                  Male                                         
2      Advanced College Degree     Male                        5' 6" (1.68 m)   
3                                  Male                                         
4                                  Male                                         
...                        ...      ...                                   ...   
28004                                                                           
28005                           Agender  40' 0" (12.19 m) (Maximum), Variable   
28006                            Female                                         
28007                              Male                                         
28008                              Male                     32' 28" (10.46 m)   

                     Weight   Eyes             Hair  \
0                            Brown            Black   
1                            Brown            Brown   
2           160 lbs (73 kg)  Brown  Black (graying)   
3                                                     
4                                             Black   
...                     ...    ...              ...   
28004       

## 1.2 Parse the Marvel comics

**Similarly to the Marvel characters, we want to first collect the comics' URLs to get to a specific comic's page. We start by the first page.**

**Collect the first URLs:**

In [ ]:
# Get URL and use html parser
URL = 'https://marvel.fandom.com/wiki/Category:Comics'
r = requests.get(URL)
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')

# Collect URL of each character in the web page
publications_wrappers = soup.find_all('li', class_='category-page__member')

# Initialization
i = 0
list_url = []

for p in publications_wrappers:
    # Don't take into account the first links which are "Categories"
    if(i>=26):
        for a in p.find_all('a', href=True):
            list_url.append(a['href'])
    i +=1
    
list_url[:20]

# Remove duplicate
my_set = set(list_url)
good_list_url = list(my_set)

**After getting the URLs, we parse all the information we need from each page. Let's note that a comic can have more than one story that are separate. We will name them `subcomic`. They will be treated as different comics since the appearing characters and the writers are often different.**

**Parse the first page:**

In [ ]:
comics_columns = ['URL', 'Good characters', 'Bad characters', 
                  'Neutral characters', 'Editor-in-chief', 'Editor-in-chief URL', 
                  'Writer', 'Writer URL', 'Publication date', 'Subcomic']

comics_pd = pd.DataFrame(columns=comics_columns)


# Parse the first page
for comics in good_list_url:
    # Get URL and use html parser
    URL_char = 'https://marvel.fandom.com' + comics
    URL_char = URL_char.replace("'","")
    r_char = requests.get(URL_char)
    page_body_char = r_char.text
    soup_char = BeautifulSoup(page_body_char, 'html.parser')
    
    # Initialization
    good, bad, neutral = '','',''
    editor, writer, publication = '','',''
    editorURL, writerURL, subcomic = '','',''
    
    URL2 = URL_char.replace('https://marvel.fandom.com','')

    # Boolean for the first case
    first = 1

    # Parse characters appearances
    appearances = soup_char.find_all('div', class_='mw-content-ltr mw-content-text')

    for p in appearances:
        for pp in p.find_all('p'):
            # If a comic is split in sub-comics
            span = pp.find_previous('span', class_='mw-headline')
            
            if span and ("Appearing" in span.text):
                if (not first and subcomic!=span.text \
                and (good or bad or neutral)):
                    # Take only the title of the subcomic
                    subcomic = subcomic[14:-1]
                    appearances_pd = pd.DataFrame([[URL2, good, bad, neutral, 
                                        editor, editorURL,
                                        writer, writerURL,
                                        publication, subcomic]], columns = comics_columns)
                    comics_pd = comics_pd.append(appearances_pd)

                    good, bad, neutral = '','',''
                    editor, writer, publication = '','',''
                    editorURL, writerURL = '',''
                    subcomic = span.text

                else:
                    first = 0
                    subcomic = span.text

            if "Featured Characters:" in pp.text:
                ul = pp.find_next('ul')
                for a in ul.find_all('a', href=True):
                    good = good + ', ' + a['href']

            if "Supporting Characters:" in pp.text:
                ul = pp.find_next('ul')
                for a in ul.find_all('a', href=True):
                    good = good + ', ' + a['href']

            if "Antagonists:" in pp.text:
                ul = pp.find_next('ul')
                for a in ul.find_all('a', href=True):
                    bad = bad + ', ' + a['href']

            if "Other Characters:" in pp.text:
                ul = pp.find_next('ul')
                for a in ul.find_all('a', href=True):
                    neutral = neutral + ', ' + a['href']

    # Each subcomic is written in the format '"Appearing in *subcomic*"'
    # We keep only the subcomic name
    subcomic = subcomic[14:-1]
    appearances_pd = pd.DataFrame([[URL2, good, bad, neutral, 
                                    editor, editorURL,
                                    writer, writerURL,
                                    publication, subcomic]], columns = comics_columns)

    comics_pd = comics_pd.append(appearances_pd)

    subcomic = ''


    # Parse comic characteristics
    side_tab = soup_char.find_all('div', class_='infobox')

    for p in side_tab:
        # Publication date
        tab = p.find_next('table', style='width:100%; text-align: center;')
        if(tab):
            tr = tab.find_next('tr', style='font-size:12px;')
            td = tab.find_next('td', style='font-size:12px;')
        if (tr):
            publication = tr.find_next('td').text
        if (td):
            publication = td.text
        if (not tr and not td):
            publication = ''

        # Editors
        for pp in p.find_all('div', style='width:100px;float:left;text-align:left;'):
            if ('Editor-in-Chief' in pp.text):
                ppp = pp.find_next('div', style='width:190px;float:left;text-align:right;')
                for a in ppp.find_all('a'):
                    editor = editor + ', ' + a.text
                for a in ppp.find_all('a', href=True):
                    editorURL = editorURL + ', ' + a['href']
                    
        if(p.find_next('div', style='width:88%; text-align:left;')):
            sub = p.find_next('div', style='width:88%; text-align:left;')

        # Writers
        for pp in p.find_all('div', style='width:100px;float:left;text-align:left;'):
            if (pp.find_previous('div', style='width:88%; text-align:left;')):
                sub = pp.find_previous('div', style='width:88%; text-align:left;')
            if sub:
                adiv = sub.find_next('div', style='width:100px;float:left;text-align:left;')

            if (adiv and 'Writer' in adiv.text and 'Writer' in pp.text):
                if (sub):
                    subcomic = sub.text[1:-1]

                ppp = pp.find_next('div', style='width:190px;float:left;text-align:right;')
                for a in ppp.find_all('a'):
                    writer = writer + ', ' + a.text
                for a in ppp.find_all('a', href=True):
                    writerURL = writerURL + ', ' + a['href']

                comics_pd.loc[comics_pd.Subcomic == subcomic,'Writer'] = writer
                comics_pd.loc[comics_pd.Subcomic == subcomic,'Writer URL'] = writerURL

                if (sub):
                    sub = sub.find_next('div', style='width:88%; text-align:left;')
                    writer, writerURL = '',''

            elif (sub): 
                sub = sub.find_next('div', style='width:88%; text-align:left;')
                writer, writerURL = '',''    
    
    comics_pd.loc[comics_pd.URL == URL2,'Editor-in-chief'] = editor
    comics_pd.loc[comics_pd.URL == URL2,'Editor-in-chief URL'] = editorURL
    comics_pd.loc[comics_pd.URL == URL2,'Publication date'] = publication

**Again, we use the next link to parse the remaining pages. The schema is exactly the same as the previous one.**

**Parse the remaining pages:**

In [ ]:
# Parse all the other pages
nextpage = soup.find('link', {"rel" : "next"})

i = 0
tot_page = 49759/200

while(len(nextpage['href'])):
    # Print a loading bar
    i += 1
    printed= i/tot_page*100
    stdout.write("\r%f %%" % printed)
    stdout.flush()
    urlnext_page = nextpage['href']
    r = requests.get(urlnext_page)
    page_body = r.text
    soup = BeautifulSoup(page_body, 'html.parser')

    # Collect url of each character in the web page
    publications_wrappers = soup.find_all('li', class_='category-page__member')
    list_url = []
    for p in publications_wrappers:
        for a in p.find_all('a', href=True):
            list_url.append(a['href'])
            
    # Remove duplicates
    my_set = set(list_url)
    good_list_url = list(my_set)
    
    for comics in good_list_url:
        # Get URL and use html parser
        URL_char = 'https://marvel.fandom.com' + comics
        URL_char = URL_char.replace("'","")
        r_char = requests.get(URL_char)
        page_body_char = r_char.text
        soup_char = BeautifulSoup(page_body_char, 'html.parser')

        # Initialization
        good, bad, neutral = '','',''
        editor, writer, publication = '','',''
        editorURL, writerURL, subcomic = '','',''

        URL2 = URL_char.replace('https://marvel.fandom.com','')

        # Boolean for the first case
        first = 1

        # Parse characters appearances
        appearances = soup_char.find_all('div', class_='mw-content-ltr mw-content-text')

        for p in appearances:
            for pp in p.find_all('p'):
                # If a comic is split in sub-comics
                span = pp.find_previous('span', class_='mw-headline')
                
                if span and ("Appearing" in span.text):
                    if (not first and subcomic!=span.text \
                    and (good or bad or neutral)):
                        # Take only the title of the subcomic
                        subcomic = subcomic[14:-1]
                        appearances_pd = pd.DataFrame([[URL2, good, bad, neutral, 
                                            editor, editorURL,
                                            writer, writerURL,
                                            publication, subcomic]], columns = comics_columns)
                        comics_pd = comics_pd.append(appearances_pd)

                        good, bad, neutral = '','',''
                        editor, writer, publication = '','',''
                        editorURL, writerURL = '',''
                        subcomic = span.text

                    else:
                        first = 0
                        subcomic = span.text

                if "Featured Characters:" in pp.text:
                    ul = pp.find_next('ul')
                    for a in ul.find_all('a', href=True):
                        good = good + ', ' + a['href']

                if "Supporting Characters:" in pp.text:
                    ul = pp.find_next('ul')
                    for a in ul.find_all('a', href=True):
                        good = good + ', ' + a['href']

                if "Antagonists:" in pp.text:
                    ul = pp.find_next('ul')
                    for a in ul.find_all('a', href=True):
                        bad = bad + ', ' + a['href']

                if "Other Characters:" in pp.text:
                    ul = pp.find_next('ul')
                    for a in ul.find_all('a', href=True):
                        neutral = neutral + ', ' + a['href']

        # Each subcomic is written in the format '"Appearing in *subcomic*"'
        # We keep only the subcomic name
        subcomic = subcomic[14:-1]
        appearances_pd = pd.DataFrame([[URL2, good, bad, neutral, 
                                        editor, editorURL,
                                        writer, writerURL,
                                        publication, subcomic]], columns = comics_columns)

        comics_pd = comics_pd.append(appearances_pd)

        subcomic = ''


        # Parse comic characteristics
        side_tab = soup_char.find_all('div', class_='infobox')

        for p in side_tab:
            # Publication date
            if(p.find_next('div', style='width:88%; text-align:left;')):
                tab = p.find_next('table', style='width:100%; text-align: center;')
            if(tab):
                tr = tab.find_next('tr', style='font-size:12px;')
                td = tab.find_next('td', style='font-size:12px;')
            if (tr):
                publication = tr.find_next('td').text
            if (td):
                publication = td.text
            if (not tr and not td):
                publication = ''

            # Editors
            for pp in p.find_all('div', style='width:100px;float:left;text-align:left;'):
                if ('Editor-in-Chief' in pp.text):
                    ppp = pp.find_next('div', style='width:190px;float:left;text-align:right;')
                    for a in ppp.find_all('a'):
                        editor = editor + ', ' + a.text
                    for a in ppp.find_all('a', href=True):
                        editorURL = editorURL + ', ' + a['href']

            sub = p.find_next('div', style='width:88%; text-align:left;')

            # Writers
            for pp in p.find_all('div', style='width:100px;float:left;text-align:left;'):
                if (pp.find_previous('div', style='width:88%; text-align:left;')):
                    sub = pp.find_previous('div', style='width:88%; text-align:left;')
                if sub:
                    adiv = sub.find_next('div', style='width:100px;float:left;text-align:left;')

                if (adiv and 'Writer' in adiv.text and 'Writer' in pp.text):
                    if (sub):
                        subcomic = sub.text[1:-1]

                    ppp = pp.find_next('div', style='width:190px;float:left;text-align:right;')
                    for a in ppp.find_all('a'):
                        writer = writer + ', ' + a.text
                    for a in ppp.find_all('a', href=True):
                        writerURL = writerURL + ', ' + a['href']

                    comics_pd.loc[comics_pd.Subcomic == subcomic,'Writer'] = writer
                    comics_pd.loc[comics_pd.Subcomic == subcomic,'Writer URL'] = writerURL

                    if (sub):
                        sub = sub.find_next('div', style='width:88%; text-align:left;')
                        writer, writerURL = '',''

                elif (sub): 
                    sub = sub.find_next('div', style='width:88%; text-align:left;')
                    writer, writerURL = '',''    

        comics_pd.loc[comics_pd.URL == URL2,'Editor-in-chief'] = editor
        comics_pd.loc[comics_pd.URL == URL2,'Editor-in-chief URL'] = editorURL
        comics_pd.loc[comics_pd.URL == URL2,'Publication date'] = publication
    nextpage = soup.find('link', {"rel" : "next"})

**Save file to pickle:**

In [ ]:
pickle.dump(comics_pd, open('comics_marvel.txt','wb'))

**Open the saved file:**

In [3]:
with open('comics_marvel.txt', 'rb') as f:
    comics_marvel = pickle.load(f)

comics_marvel

URL  \
0                /wiki/Marvel_Mystery_Comics_Vol_1_NN   
0                        /wiki/Comedy_Comics_Vol_1_12   
0                 /wiki/Marvel_Mystery_Comics_Vol_1_7   
0                 /wiki/Marvel_Mystery_Comics_Vol_1_7   
0                 /wiki/Marvel_Mystery_Comics_Vol_1_7   
..                                                ...   
0   /wiki/Spider-Man:_The_Complete_Clone_Saga_Epic...   
0   /wiki/Spider-Man:_The_Complete_Clone_Saga_Epic...   
0                           /wiki/Hellraiser_Vol_1_17   
0   /wiki/Ultimate_Spider-Man_Infinite_Comic_Vol_2_10   
0   /wiki/Marvel_Universe_Ultimate_Spider-Man_vs._...   

                                      Good characters  \
0                                                       
0                                                       
0   , /wiki/Human_Torch_(Android)_(Earth-616), /wi...   
0   , /wiki/Thomas_Halloway_(Earth-616), /wiki/Bet...   
0   , /wiki/Namor_McKenzie_(Earth-616), /wiki/Thak...   
..                                                ...   
0   , /wiki/Peter_Parker_(Earth-616), /wiki/Ben_Re...   
0   , /wiki/Peter_Parker_(Earth-616), /wiki/Ben_Re...   
0                                                       
0   , /wiki/Peter_Parker_(Earth-12041), /wiki/Pete...   
0   , /wiki/Web-Warriors_(Earth-12041), /wiki/Pete...   

                                       Bad characters  \
0                                                       
0                                                       
0   , /wiki/Roglo_(Earth-616), #cite_note-Only_App...   
0                     , /wiki/Emma_Martin_(Earth-616)   
0                                                       
..                                                ...   
0   , /wiki/Kaine_Parker_(Earth-616), /wiki/Samuel...   
0   , /wiki/Miles_Warren_(Jackal_Clone_2)_(Earth-616)   
0                                                       
0                       , /wiki/Shazana_(Earth-12041)   
0   , /wiki/Norman_Osborn_(Earth-TRN457), /wiki/Hy...   

                                   Neutral characters Editor-in-chief  \
0                                                                       
0                                                          , Stan Lee   
0   , /wiki/New_York_City_Police_Department_(Earth...     , Joe Simon   
0                    , /wiki/Henry_Martin_(Earth-616)     , Joe Simon   
0   , /wiki/Homo_mermanus, /wiki/New_York_City_Pol...     , Joe Simon   
..                                                ...             ...   
0   , /wiki/Guardian_(Spider-Clone)_(Earth-616), /...   , Joe Quesada   
0   , /wiki/Kaine_Parker_(Earth-616), /wiki/Charle...   , Joe Quesada   
0                                                                       
0   , /wiki/William_Howard_Taft_(Earth-12041), /wi...   , Axel Alonso   
0   , /wiki/Rio_Morales_(Earth-TRN457), /wiki/Jeff...   , Axel Alonso   

    Editor-in-chief URL                      Writer  \
0                                   , Joe Caramagna   
0      , /wiki/Stan_Lee                               
0     , /wiki/Joe_Simon    , Stan Lee, Larry Lieber   
0     , /wiki/Joe_Simon  , Paul Gustavson, Ray Gill   
0     , /wiki/Joe_Simon     , William Blake Everett   
..                  ...                         ...   
0   , /wiki/Joe_Quesada            , J.M. DeMatteis   
0   , /wiki/Joe_Quesada             , Howard Mackie   
0                                    , Clive Barker   
0   , /wiki/Axel_Alonso               , John Barber   
0   , /wiki/Axel_Alonso             , Joe Caramagna   

                                Writer URL Publication date  \
0                    , /wiki/Joe_Caramagna    January, 1943   
0                                            December, 1942   
0     , /wiki/Stan_Lee, /wiki/Larry_Lieber        May, 1940   
0   , /wiki/Paul_Gustavson, /wiki/Ray_Gill        May, 1940   
0            , /wiki/William_Blake_Everett        May, 1940   
..                                     ...              ...   
0               

# Part 2: DC dataset

**Let's note that the DC dataset is extremely similar to the Marvel dataset. Indeed, the characters have the same characteristics and we can thus use the same attributes. The difference is that the characters are in two different pages which are classified into good and bad characters (compared to the Marvel dataset where we took all the characters in `Earth-616`, not knowing if they were good or bad). Moreover, the comics are also very similarly written and we can apply a similar code as the one we did for Marvel. We just have to be careful when we use the different `find` functions. Indeed, even if the characteristics are the same, the html code might be different and we had to adapt the code we had.**

## 2.1 Parse the DC characters:

### 2.1.1 Good DC characters

**Collect the first URLs:**

In [ ]:
URL = 'https://dc.fandom.com/wiki/Category:Good_Characters'
r = requests.get(URL)
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')

# Collect URL of each character in the web page
publications_wrappers = soup.find_all('li', class_='category-page__member')

list_url = []
for p in publications_wrappers:
    for a in p.find_all('a', href=True):
        list_url.append(a['href'])
        
# Remove duplicate
my_set = set(list_url)
good_list_url = list(my_set)

**Parse the first pages:**

In [ ]:
new_columns = ['URL', 'Real Name', 'Identity', 'Current Alias', 'Citizenship',
                'Marital Status', 'Occupation',
                'Education', 'Gender', 'Height', 'Weight',
                'Eyes','Hair',
                'Place of Birth']
personnage_pd = pd.DataFrame(columns=new_columns)
idx = 0
dict_geant = {}
for pers in good_list_url:
    # Get URL and use html parser
    URL_char = 'https://dc.fandom.com/' + pers
    URL_char = URL_char.replace("'","")
    r_char = requests.get(URL_char)
    page_body_char = r_char.text
    soup_char = BeautifulSoup(page_body_char, 'html.parser')
    
    # Initialize variables
    url, name, identity, current_alias, citizenship, marital_status, occupation = '','','','','','',''
    education, gender, height, weight, eyes, hair, place_of_birth = '','','','','','',''
    
    personnage =[]
    
    # Parsing
    url = URL_char
    side_tab = soup_char.find_all('div', class_='mw-content-ltr mw-content-text')
    for p in side_tab:
        for pp in p.find_all('div', class_='pi-item pi-data pi-item-spacing pi-border-color'):
            for ppp in pp.find_all('h3', class_='pi-data-label pi-secondary-font'):
                if(ppp.text[0:]=="Real Name"):
                    name = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Identity"):
                    identity = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Current Alias"):
                    current_alias = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Citizenship"):
                    citizenship = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Marital Status"):
                    marital_status = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Occupation"):
                    occupation = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Education"):
                    education = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Gender"):
                    gender = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Height"):
                    height = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Weight"):
                    weight = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Eyes"):
                    eyes = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Hair"):
                    hair = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Place of Birth"):
                    place_of_birth = pp.find('div', class_='pi-data-value pi-font').text

        characteristics_pd = pd.DataFrame([[url, name[0:], identity, current_alias, citizenship, marital_status, occupation,
                          education, gender, height, weight, eyes, hair, place_of_birth]], columns = new_columns)

        personnage_pd = personnage_pd.append(characteristics_pd,ignore_index=True)
    
nextpage = soup.find('link', {"rel" : "next"})

**Parse the remaining pages:**

In [ ]:
while(len(nextpage['href'])):
    urlnext_page = nextpage['href']
    r = requests.get(urlnext_page)
    page_body = r.text
    soup = BeautifulSoup(page_body, 'html.parser')

    # Collect URL of each character in the web page
    publications_wrappers = soup.find_all('li', class_='category-page__member')
    list_url = []
    for p in publications_wrappers:
        for a in p.find_all('a', href=True):
            list_url.append(a['href'])
            
    # Remove duplicate
    my_set =set(list_url)
    good_list_url = list(my_set)
    idx =0
    dict_geant={}
    for pers in good_list_url:
        # Get URL and use html parser
        URL_char = 'https://dc.fandom.com/' + pers
        URL_char = URL_char.replace("'","")
        r_char = requests.get(URL_char)
        page_body_char = r_char.text
        soup_char = BeautifulSoup(page_body_char, 'html.parser')
        
    # Initialize variables
    url, name, identity, current_alias, citizenship, marital_status, occupation = '','','','','','',''
    education, gender, height, weight, eyes, hair, place_of_birth = '','','','','','',''
    
    personnage =[]
    
    # Parsing
    url = URL_char
    side_tab = soup_char.find_all('div', class_='mw-content-ltr mw-content-text')
    for p in side_tab:
        for pp in p.find_all('div', class_='pi-item pi-data pi-item-spacing pi-border-color'):
            for ppp in pp.find_all('h3', class_='pi-data-label pi-secondary-font'):
            if(ppp.text[0:]=="Real Name"):
                name = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Identity"):
                identity = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Current Alias"):
                current_alias = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Citizenship"):
                citizenship = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Marital Status"):
                marital_status = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Occupation"):
                occupation = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Education"):
                education = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Gender"):
                gender = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Height"):
                height = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Weight"):
                weight = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Eyes"):
                eyes = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Hair"):
                hair = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Place of Birth"):
                place_of_birth = pp.find('div', class_='pi-data-value pi-font').text

    characteristics_pd = pd.DataFrame([[url, name[0:], identity, current_alias, citizenship, marital_status, occupation,
                      education, gender, height, weight, eyes, hair, place_of_birth]], columns = new_columns)

    personnage_pd = personnage_pd.append(characteristics_pd,ignore_index=True)
    
    nextpage = soup.find('link', {"rel" : "next"})
    
# Create a column to classify the characters as good ones
personnage_pd.insert(5,'Good or Bad', "Good", True)

### 2.1.2 Bad DC characters

**Collect the first URLs:**

In [ ]:
URL = 'https://dc.fandom.com/wiki/Category:Bad_Characters'
r = requests.get(URL)
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')

# Collect URL of each character in the web page
publications_wrappers = soup.find_all('li', class_='category-page__member')

list_url = []
for p in publications_wrappers:
    for a in p.find_all('a', href=True):
        list_url.append(a['href'])
        
# Remove duplicate
my_set = set(list_url)
good_list_url = list(my_set)

**Parse the first pages:**

In [ ]:
new_columns = ['URL', 'Real Name', 'Identity', 'Current Alias', 'Citizenship',
                'Marital Status', 'Occupation',
                'Education', 'Gender', 'Height', 'Weight',
                'Eyes','Hair',
                'Place of Birth']
bad_pers_dc = pd.DataFrame(columns=new_columns)
idx = 0
dict_geant = {}
for pers in good_list_url:
    # Get URL and use html parser
    URL_char = 'https://dc.fandom.com/' + pers
    URL_char = URL_char.replace("'","")
    r_char = requests.get(URL_char)
    page_body_char = r_char.text
    soup_char = BeautifulSoup(page_body_char, 'html.parser')
    
    # Initialize variables
    url, name, identity, current_alias, citizenship, marital_status, occupation = '','','','','','',''
    education, gender, height, weight, eyes, hair, place_of_birth = '','','','','','',''
    
    personnage =[]
    
    # Parsing
    url = URL_char
    side_tab = soup_char.find_all('div', class_='mw-content-ltr mw-content-text')
    for p in side_tab:
        for pp in p.find_all('div', class_='pi-item pi-data pi-item-spacing pi-border-color'):
            for ppp in pp.find_all('h3', class_='pi-data-label pi-secondary-font'):
                if(ppp.text[0:]=="Real Name"):
                    name = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Identity"):
                    identity = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Current Alias"):
                    current_alias = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Citizenship"):
                    citizenship = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Marital Status"):
                    marital_status = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Occupation"):
                    occupation = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Education"):
                    education = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Gender"):
                    gender = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Height"):
                    height = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Weight"):
                    weight = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Eyes"):
                    eyes = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Hair"):
                    hair = pp.find('div', class_='pi-data-value pi-font').text
                if(ppp.text[0:]=="Place of Birth"):
                    place_of_birth = pp.find('div', class_='pi-data-value pi-font').text

        characteristics_pd = pd.DataFrame([[url, name[0:], identity, current_alias, citizenship, marital_status, occupation,
                          education, gender, height, weight, eyes, hair, place_of_birth]], columns = new_columns)

        bad_pers_dc = bad_pers_dc.append(characteristics_pd,ignore_index=True)
    
nextpage = soup.find('link', {"rel" : "next"})

**Parse the remaining pages:**

In [ ]:
while(len(nextpage['href'])):
    urlnext_page = nextpage['href']
    r = requests.get(urlnext_page)
    page_body = r.text
    soup = BeautifulSoup(page_body, 'html.parser')

    # Collect URL of each character in the web page
    publications_wrappers = soup.find_all('li', class_='category-page__member')
    list_url = []
    for p in publications_wrappers:
        for a in p.find_all('a', href=True):
            list_url.append(a['href'])
            
    # Remove duplicate
    my_set =set(list_url)
    good_list_url = list(my_set)
    idx =0
    dict_geant={}
    for pers in good_list_url:
        # Get URL and use html parser
        URL_char = 'https://dc.fandom.com/' + pers
        URL_char = URL_char.replace("'","")
        r_char = requests.get(URL_char)
        page_body_char = r_char.text
        soup_char = BeautifulSoup(page_body_char, 'html.parser')
        
    # Initialize variables
    url, name, identity, current_alias, citizenship, marital_status, occupation = '','','','','','',''
    education, gender, height, weight, eyes, hair, place_of_birth = '','','','','','',''
    
    personnage =[]
    
    # Parsing
    url = URL_char
    side_tab = soup_char.find_all('div', class_='mw-content-ltr mw-content-text')
    for p in side_tab:
        for pp in p.find_all('div', class_='pi-item pi-data pi-item-spacing pi-border-color'):
            for ppp in pp.find_all('h3', class_='pi-data-label pi-secondary-font'):
            if(ppp.text[0:]=="Real Name"):
                name = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Identity"):
                identity = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Current Alias"):
                current_alias = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Citizenship"):
                citizenship = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Marital Status"):
                marital_status = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Occupation"):
                occupation = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Education"):
                education = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Gender"):
                gender = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Height"):
                height = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Weight"):
                weight = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Eyes"):
                eyes = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Hair"):
                hair = pp.find('div', class_='pi-data-value pi-font').text
            if(ppp.text[0:]=="Place of Birth"):
                place_of_birth = pp.find('div', class_='pi-data-value pi-font').text

    characteristics_pd = pd.DataFrame([[url, name[0:], identity, current_alias, citizenship, marital_status, occupation,
                      education, gender, height, weight, eyes, hair, place_of_birth]], columns = new_columns)

    bad_pers_dc = bad_pers_dc.append(characteristics_pd,ignore_index=True)
    
    nextpage = soup.find('link', {"rel" : "next"})
    
# Create a column to classify the characters as bad ones
personnage_pd.insert(5,'Good or Bad', "Bad", True)

### 2.1.3 Join dataframes

**Concatenate the dataframes for good and bad characters:**

In [ ]:
pers_dc = [personnage_pd, bad_pers_dc]
pd.concat(pers_dc)

**Save to pickle:**

In [ ]:
pickle.dump(pers_dc, open('perso_dc.txt','wb'))

**Open the saved file:**

In [ ]:
character_dc = pd.read_pickle('perso_dc.txt')

character_dc

# 2.2 Parse the DC comics:

**Collect the first URLs:**

In [ ]:
# Get URL and use html parser
URL = 'https://dc.fandom.com/wiki/Category:Comics'
r = requests.get(URL)
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')

# Collect URL of each character in the web page
publications_wrappers = soup.find_all('li', class_='category-page__member')

# Initialization
i = 0
list_url = []

for p in publications_wrappers:
    # Don't take into account the first links which are "Categories"
    if(i>=24):
        for a in p.find_all('a', href=True):
            list_url.append(a['href'])
    i +=1
    
# Remove duplicate
my_set = set(list_url)
good_list_url = list(my_set)

**Parse the first pages:**

In [ ]:
comics_columns = ['URL', 'Good characters', 'Bad characters', 
                  'Neutral characters', 'Editor-in-chief', 'Editor-in-chief URL', 
                  'Writer', 'Writer URL', 'Publication date', 'Subcomic']

comics_pd = pd.DataFrame(columns=comics_columns)

# Parse the first page
for comics in good_list_url:
    # Get URL and use html parser
    URL_char = 'https://dc.fandom.com' + comics
    URL_char = URL_char.replace("'","")
    r_char = requests.get(URL_char)
    page_body_char = r_char.text
    soup_char = BeautifulSoup(page_body_char, 'html.parser')
    
    # Initialization
    good, bad, neutral = '','',''
    editor, writer, publication = '','',''
    editorURL, writerURL, subcomic = '','',''
    URL2 = URL_char.replace('https://dc.fandom.com','')
    
    # Boolean for the first case
    first = 1
    
    # Parse characters appearances
    appearances = soup_char.find_all('div', class_='mw-content-ltr mw-content-text')
    for p in appearances:
        for pp in p.find_all('p'):
            # If a comic is split in sub-comics
            span = pp.find_previous('span', class_='mw-headline')
            if span and ("Appearing" in span.text):
                if (not first and subcomic!=span.text \
                and (good or bad or neutral)):
                    # Take only the title of the subcomic
                    subcomic = subcomic[14:-1]
                    appearances_pd = pd.DataFrame([[URL2, good, bad, neutral, 
                                        editor, editorURL,
                                        writer, writerURL,
                                        publication, subcomic]], columns = comics_columns)
                    comics_pd = comics_pd.append(appearances_pd)
                    good, bad, neutral = '','',''
                    editor, writer, publication = '','',''
                    editorURL, writerURL = '',''
                    subcomic = span.text
                else:
                    first = 0
                    subcomic = span.text
                    
            if "Featured Characters:" in pp.text:
                ul = pp.find_next('ul')
                for a in ul.find_all('a', href=True):
                    good = good + ', ' + a['href']
            if "Supporting Characters:" in pp.text:
                ul = pp.find_next('ul')
                for a in ul.find_all('a', href=True):
                    good = good + ', ' + a['href']
            if "Antagonists:" in pp.text:
                ul = pp.find_next('ul')
                for a in ul.find_all('a', href=True):
                    bad = bad + ', ' + a['href']
            if "Other Characters:" in pp.text:
                ul = pp.find_next('ul')
                for a in ul.find_all('a', href=True):
                    neutral = neutral + ', ' + a['href']
                    
    # Each subcomic is written in the format '"Appearing in *subcomic*"'
    # We keep only the subcomic name
    subcomic = subcomic[14:-1]
    appearances_pd = pd.DataFrame([[URL2, good, bad, neutral, 
                                    editor, editorURL,
                                    writer, writerURL,
                                    publication, subcomic]], columns = comics_columns)
    comics_pd = comics_pd.append(appearances_pd)
    subcomic = ''
    
    
    # Parse comic characteristics
    side_tab = soup_char.find_all('div', class_='mw-content-ltr mw-content-text')
    for p in side_tab:
        # Publication date
        if(p.find_next('h2', {'data-source':'StoryTitle1'})):
            tab = p.find_next('h2', class_='pi-item pi-item-spacing pi-title')
        if(tab):
            tab = tab.find_next('h2', class_='pi-item pi-item-spacing pi-title')
        if(tab):
            tab = tab.find_next('h2', class_='pi-item pi-item-spacing pi-title')
        if(tab):
            publication = tab.text
            
        # Editors
        for pp in p.find_all('h3', class_='pi-data-label pi-secondary-font'):
            if ('Executive Editor' in pp.text):
                ppp = pp.find_next('div', class_='pi-data-value pi-font')
                for a in ppp.find_all('a'):
                    editor = editor + ', ' + a.text
                for a in ppp.find_all('a', href=True):
                    editorURL = editorURL + ', ' + a['href']

        sub = tab.find_next('h2', class_='pi-item pi-header pi-secondary-font pi-item-spacing pi-secondary-background')

        # Writers
        for pp in p.find_all('h3', class_='pi-data-label pi-secondary-font'):
            if(pp.find_previous('h2', class_='pi-item pi-header pi-secondary-font pi-item-spacing pi-secondary-background')):
                sub = pp.find_previous('h2', class_='pi-item pi-header pi-secondary-font pi-item-spacing pi-secondary-background')
            if sub:
                adiv = sub.find_next('h3', class_='pi-data-label pi-secondary-font')
            if (adiv and 'Writer' in adiv.text and 'Writer' in pp.text):
                if (sub):
                    subcomic = sub.text[1:-1]
                ppp = pp.find_next('div', class_='pi-data-value pi-font')
                for a in ppp.find_all('a'):
                    writer = writer + ', ' + a.text
                for a in ppp.find_all('a', href=True):
                    writerURL = writerURL + ', ' + a['href']
                comics_pd.loc[comics_pd.Subcomic == subcomic,'Writer'] = writer
                comics_pd.loc[comics_pd.Subcomic == subcomic,'Writer URL'] = writerURL
                if (sub):
                    sub = sub.find_next('h2', class_='pi-item pi-header pi-secondary-font pi-item-spacing pi-secondary-background')
                    writer, writerURL = '',''
            elif (sub): 
                sub = sub.find_next('h2', class_='pi-item pi-header pi-secondary-font pi-item-spacing pi-secondary-background')
                writer, writerURL = '',''    
                
    comics_pd.loc[comics_pd.URL == URL2,'Editor-in-chief'] = editor
    comics_pd.loc[comics_pd.URL == URL2,'Editor-in-chief URL'] = editorURL
    comics_pd.loc[comics_pd.URL == URL2,'Publication date'] = publication

**Parse the remaining pages:**

In [ ]:
# Parse all the other pages
nextpage = soup.find('link', {"rel" : "next"})

i = 0
tot_page = 49759/200

while(len(nextpage['href'])):
    # Print a loading bar
    i += 1
    printed= i/tot_page*100
    stdout.write("\r%f %%" % printed)
    stdout.flush()
    urlnext_page = nextpage['href']
    r = requests.get(urlnext_page)
    page_body = r.text
    soup = BeautifulSoup(page_body, 'html.parser')
    
    # Collect url of each character in the web page
    publications_wrappers = soup.find_all('li', class_='category-page__member')
    list_url = []
    for p in publications_wrappers:
        for a in p.find_all('a', href=True):
            list_url.append(a['href'])
          
    # Remove duplicates
    my_set = set(list_url)
    good_list_url = list(my_set)
    good_list_url = [ x for x in good_list_url if "/wiki/Category:" not in x ]
  
    # Parse the first page
    for comics in good_list_url:
        # Get URL and use html parser
        URL_char = 'https://dc.fandom.com' + comics
        URL_char = URL_char.replace("'","")
        r_char = requests.get(URL_char)
        page_body_char = r_char.text
        soup_char = BeautifulSoup(page_body_char, 'html.parser')
        
        # Initialization
        good, bad, neutral = '','',''
        editor, writer, publication = '','',''
        editorURL, writerURL, subcomic = '','',''
        URL2 = URL_char.replace('https://dc.fandom.com','')

        # Boolean for the first case
        first = 1

        # Parse characters appearances
        appearances = soup_char.find_all('div', class_='mw-content-ltr mw-content-text')
        for p in appearances:
            for pp in p.find_all('p'):
                # If a comic is split in sub-comics
                span = pp.find_previous('span', class_='mw-headline')
                if span and ("Appearing" in span.text):
                    if (not first and subcomic!=span.text \
                    and (good or bad or neutral)):
                        # Take only the title of the subcomic
                        subcomic = subcomic[14:-1]
                        appearances_pd = pd.DataFrame([[URL2, good, bad, neutral, 
                                              editor, editorURL,
                                              writer, writerURL,
                                              publication, subcomic]], columns = comics_columns)
                        comics_pd = comics_pd.append(appearances_pd)

                        good, bad, neutral = '','',''
                        editor, writer, publication = '','',''
                        editorURL, writerURL = '',''
                        subcomic = span.text
                    else:
                        first = 0
                        subcomic = span.text
                if "Featured Characters:" in pp.text:
                    ul = pp.find_next('ul')
                    for a in ul.find_all('a', href=True):
                        good = good + ', ' + a['href']
                if "Supporting Characters:" in pp.text:
                    ul = pp.find_next('ul')
                    for a in ul.find_all('a', href=True):
                        good = good + ', ' + a['href']
                if "Antagonists:" in pp.text:
                    ul = pp.find_next('ul')
                    for a in ul.find_all('a', href=True):
                        bad = bad + ', ' + a['href']
                if "Other Characters:" in pp.text:
                    ul = pp.find_next('ul')
                    for a in ul.find_all('a', href=True):
                        neutral = neutral + ', ' + a['href']

        # Each subcomic is written in the format '"Appearing in *subcomic*"'
        # We keep only the subcomic name
        subcomic = subcomic[14:-1]

        appearances_pd = pd.DataFrame([[URL2, good, bad, neutral, 
                                          editor, editorURL,
                                          writer, writerURL,
                                          publication, subcomic]], columns = comics_columns)

        comics_pd = comics_pd.append(appearances_pd)

        subcomic = ''


        # Parse comic characteristics
        side_tab = soup_char.find_all('div', class_='mw-content-ltr mw-content-text')
        for p in side_tab:
            # Publication date
            if(p.find_next('h2', {'data-source':'StoryTitle1'})):
                tab = p.find_next('h2', class_='pi-item pi-item-spacing pi-title')
                if(tab):
                    tab = tab.find_next('h2', class_='pi-item pi-item-spacing pi-title')
                if(tab):
                    tab = tab.find_next('h2', class_='pi-item pi-item-spacing pi-title')
                if(tab):
                    publication = tab.text
            else:
                if(tab):
                    tab = p.find_next('h2', class_='pi-item pi-item-spacing pi-title')
                if(tab):
                    tab = tab.find_next('h2', class_='pi-item pi-item-spacing pi-title')
                if(tab):
                    publication = tab.text

            # Editors
            for pp in p.find_all('h3', class_='pi-data-label pi-secondary-font'):
                if ('Executive Editor' in pp.text):
                    ppp = pp.find_next('div', class_='pi-data-value pi-font')
                    for a in ppp.find_all('a'):
                        editor = editor + ', ' + a.text
                    for a in ppp.find_all('a', href=True):
                        editorURL = editorURL + ', ' + a['href']
            if(tab):           
                if(tab.find_next('h2', class_='pi-item pi-header pi-secondary-font pi-item-spacing pi-secondary-background')):
                      sub = tab.find_next('h2', class_='pi-item pi-header pi-secondary-font pi-item-spacing pi-secondary-background')

            # Writers
            for pp in p.find_all('h3', class_='pi-data-label pi-secondary-font'):
                if(pp.find_previous('h2', class_='pi-item pi-header pi-secondary-font pi-item-spacing pi-secondary-background')):
                    sub = pp.find_previous('h2', class_='pi-item pi-header pi-secondary-font pi-item-spacing pi-secondary-background')
                if sub:
                    adiv = sub.find_next('h3', class_='pi-data-label pi-secondary-font')
                if (adiv and 'Writer' in adiv.text and 'Writer' in pp.text):
                    if (sub):
                        subcomic = sub.text[1:-1]
                        ppp = pp.find_next('div', class_='pi-data-value pi-font')
                    for a in ppp.find_all('a'):
                        writer = writer + ', ' + a.text
                    for a in ppp.find_all('a', href=True):
                        writerURL = writerURL + ', ' + a['href']
                    comics_pd.loc[comics_pd.Subcomic == subcomic,'Writer'] = writer
                    comics_pd.loc[comics_pd.Subcomic == subcomic,'Writer URL'] = writerURL
                    if (sub):
                        sub = sub.find_next('h2', class_='pi-item pi-header pi-secondary-font pi-item-spacing pi-secondary-background')
                        writer, writerURL = '',''
                elif (sub): 
                    sub = sub.find_next('h2', class_='pi-item pi-header pi-secondary-font pi-item-spacing pi-secondary-background')
                    writer, writerURL = '',''    
                
        comics_pd.loc[comics_pd.URL == URL2,'Editor-in-chief'] = editor
        comics_pd.loc[comics_pd.URL == URL2,'Editor-in-chief URL'] = editorURL
        comics_pd.loc[comics_pd.URL == URL2,'Publication date'] = publication
    nextpage = soup.find('link', {"rel" : "next"})


**Save to pickle:**

In [ ]:
pickle.dump(comics_pd, open('characters_dc.txt','wb'))

**Open the saved file:**

In [11]:
with open('comics_dc.txt', 'rb') as f:
    comics_pd = pickle.load(f)

comics_pd

URL  \
0      /wiki/100_Bullets_Vol_1_64   
0      /wiki/100_Bullets_Vol_1_25   
0      /wiki/2020_Visions_Vol_1_5   
0    /wiki/100%25_True%3F_Vol_1_2   
0      /wiki/100_Bullets_Vol_1_11   
..                            ...   
0           /wiki/Zatanna_Vol_2_1   
0         /wiki/Zero_Girl_Vol_1_4   
0   /wiki/Young_Romance_Vol_1_196   
0   /wiki/Young_Romance_Vol_1_126   
0   /wiki/Young_Romance_Vol_1_200   

                                      Good characters Bad characters  \
0   , /wiki/Jack_Daw_(100_Bullets), /wiki/Philip_G...                  
0   , /wiki/Augustus_Medici_(100_Bullets), /wiki/B...                  
0                                                                      
0                                                                      
0                 , /wiki/Philip_Graves_(100_Bullets)                  
..                                                ...            ...   
0                                                                      
0                                                                      
0                                                                      
0                                                                      
0                                                                      

   Neutral characters Editor-in-chief   Editor-in-chief URL      Writer  \
0                      , Karen Berger  , /wiki/Karen_Berger               
0                      , Karen Berger  , /wiki/Karen_Berger               
0                      , Karen Berger  , /wiki/Karen_Berger  , Ron Marz   
0                      , Jenette Kahn  , /wiki/Jenette_Kahn  , Ron Marz   
0                      , Karen Berger  , /wiki/Karen_Berger               
..                ...             ...                   ...         ...   
0                         , Dan DiDio     , /wiki/Dan_DiDio               
0                           , Jim Lee       , /wiki/Jim_Lee               
0                                                                         
0                                                                         
0                                                                         

          Writer URL Publication date                            Subcomic  
0                      November, 2005                            The Dive  
0                        August, 2001  Red Prince Blues (Part III of III)  
0   , /wiki/Ron_Marz  September, 1997                                      
0   , /wiki/Ron_Marz   December, 1997                                      
0                          June, 2000           Heartbreak, Sunny Side Up  
..               ...              ...                                 ...  
0                          July, 2010                                      
0                           May, 2001                                      
0                      December, 1973                         he 1st Stor  
0                      November, 1963                         he 1st Stor  
0                        August, 1974                         he 1st Stor  

[62314 rows x 10 columns]